# Quora similarity calculation

In [56]:
import numpy as np 
import pandas as pd 
import os

import seaborn as sns
import pandas as pd

## Import the dataset from the Desktop Location

In [58]:
train=pd.read_csv('Desktop/train.csv')

In [59]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Removing Punctuation

In [60]:
# Remove Punctuation
def punc(df,stri):
    df[stri] = df[stri].str.lower().str.replace('[^a-z]', ' ')
    df[stri] = df[stri].str.lower().str.replace('[^\w\s]',' ')
    df[stri] = df[stri].str.lower().str.replace(r'<.*?>',' ')
    df[stri] = df[stri].str.lower().str.replace(' br ',' ')
    return df

In [61]:
train=punc(train,"question1")
train=punc(train,"question2")

In [62]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math math i...,0
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0


In [64]:
train=train.dropna()

## Removing stopwords

In [65]:
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')
def stopi(df,stri): 
    df[stri] = df[stri].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    return df

In [66]:
train=stopi(train,"question1")
train=stopi(train,"question2")

In [67]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,step step guide invest share market india,step step guide invest share market,0
1,1,3,4,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,0
2,2,5,6,increase speed internet connection using vpn,internet speed increased hacking dns,0
3,3,7,8,mentally lonely solve,find remainder math math divided,0
4,4,9,10,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0


## Stemming words with nltk
Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”.

In [68]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
def port(df,stri):
    df[stri] = df[stri].apply(lambda x: " ".join(ps.stem(word) for word in x.split()))
    return df

train=port(train,"question1")
train=port(train,"question2")

## Lemmatization

In [69]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
train['question1'] = train['question1'].apply(lambda x: " ".join(wordnet_lemmatizer.lemmatize(word) for word in x.split()))
train['question2'] = train['question2'].apply(lambda x: " ".join(wordnet_lemmatizer.lemmatize(word) for word in x.split()))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\praneeth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [70]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,step step guid invest share market india,step step guid invest share market,0
1,1,3,4,stori kohinoor koh noor diamond,would happen indian govern stole kohinoor koh ...,0
2,2,5,6,increas speed internet connect use vpn,internet speed increas hack dn,0
3,3,7,8,mental lone solv,find remaind math math divid,0
4,4,9,10,one dissolv water quikli sugar salt methan car...,fish would surviv salt water,0


## SequenceMatcher to compare both the questions

In [71]:
from difflib import SequenceMatcher
a = "Dump Administration Dismisses Surgeon General Vivek Murthy (http)PUGheO7BuT5LUEtHDcgm"
b = "Dump Administration Dismisses Surgeon General Vivek Murthy (http)avGqdhRVOO"
ratio = SequenceMatcher(None, a, b).ratio()
ar=[]
for index, row in train.iterrows():
    #print(row['c1'], row['c2'])
    a=SequenceMatcher(None,row['question1'],row['question2']).ratio()
    ar.append(a)

In [72]:
train['similarity']=ar
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,similarity
0,0,1,2,step step guid invest share market india,step step guid invest share market,0,0.918919
1,1,3,4,stori kohinoor koh noor diamond,would happen indian govern stole kohinoor koh ...,0,0.617021
2,2,5,6,increas speed internet connect use vpn,internet speed increas hack dn,0,0.441176
3,3,7,8,mental lone solv,find remaind math math divid,0,0.272727
4,4,9,10,one dissolv water quikli sugar salt methan car...,fish would surviv salt water,0,0.235294
